In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
from sklearn.decomposition import PCA
from kmodes.kmodes import KModes
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
# Read OrdonezA sensor events dataset
#df_a_desc = pd.read_csv("/Users/vpriyanca/Downloads/MLDL course/UCI excel dataset /OrdonezA_Description.csv")
df_a_sensors = pd.read_csv("/Users/vpriyanca/Downloads/MLDL course/ML DL Project 2/Imported data/OrdonezA_Sensors.csv")
df_a_labels = pd.read_csv("/Users/vpriyanca/Downloads/MLDL course/ML DL Project 2/Imported data/OrdonezA_ADLs.csv")
#df_a_desc.head()
#print (df_a_sensors.head())
print(df_a_labels.head())

            Start time             End time   Activity
0  2011-11-28 02:27:00  2011-11-28 10:18:00   Sleeping
1  2011-11-28 10:21:00  2011-11-28 10:23:00  Toileting
2  2011-11-28 10:25:00  2011-11-28 10:33:00  Showering
3  2011-11-28 10:34:00  2011-11-28 10:43:00  Breakfast
4  2011-11-28 10:49:00  2011-11-28 10:51:00   Grooming


In [3]:
# Read OrdonezB sensor events dataset
#df_b_desc = pd.read_csv("/Users/vpriyanca/Downloads/MLDL course/UCI excel dataset /OrdonezB_Description.csv")
df_b_sensors = pd.read_csv("/Users/vpriyanca/Downloads/MLDL course/ML DL Project 2/Imported data/OrdonezB_Sensors.csv")
df_b_labels = pd.read_csv("/Users/vpriyanca/Downloads/MLDL course/ML DL Project 2/Imported data/OrdonezB_ADLs.csv")
#df_b_desc.head()
print (df_b_sensors.head())
#print(df_b_labels.head())

            Start time             End time Location      Type    Place
0  2012-11-11 21:14:21  2012-11-12 00:21:49     Seat  Pressure   Living
1  2012-11-12 00:22:57  2012-11-12 00:22:59     Door       PIR   Living
2  2012-11-12 00:23:14  2012-11-12 00:23:17     Door       PIR  Kitchen
3  2012-11-12 00:24:20  2012-11-12 00:24:22     Door       PIR  Kitchen
4  2012-11-12 00:24:42  2012-11-12 00:24:54     Door       PIR   Living


In [4]:
# Merge the two DataFrames on the 'Start time' and 'End time' columns using an outer join
df_merge_labels = pd.merge(df_a_labels, df_b_labels, on=['Start time', 'End time'], how='outer')
df_merge_sensors = pd.merge(df_a_sensors, df_b_sensors, on=['Start time', 'End time'], how='outer')
# Print the first few rows of the merged DataFrame
print(df_merge_labels.head())
print(df_merge_sensors.head())
#df_merge.isnull().sum()
#df_merge.duplicated().count()
#df_merge.dropna(inplace=True)
df_merge.shape

            Start time             End time Activity_x Activity_y
0  2011-11-28 02:27:00  2011-11-28 10:18:00   Sleeping        NaN
1  2011-11-28 10:21:00  2011-11-28 10:23:00  Toileting        NaN
2  2011-11-28 10:25:00  2011-11-28 10:33:00  Showering        NaN
3  2011-11-28 10:34:00  2011-11-28 10:43:00  Breakfast        NaN
4  2011-11-28 10:49:00  2011-11-28 10:51:00   Grooming        NaN
            Start time             End time Location_x    Type_x   Place_x  \
0  2011-11-28 02:27:59  2011-11-28 10:18:11        Bed  Pressure   Bedroom   
1  2011-11-28 10:21:24  2011-11-28 10:21:31    Cabinet  Magnetic  Bathroom   
2  2011-11-28 10:21:44  2011-11-28 10:23:31      Basin       PIR  Bathroom   
3  2011-11-28 10:23:02  2011-11-28 10:23:36     Toilet     Flush  Bathroom   
4  2011-11-28 10:25:44  2011-11-28 10:32:06     Shower       PIR  Bathroom   

  Location_y Type_y Place_y  
0        NaN    NaN     NaN  
1        NaN    NaN     NaN  
2        NaN    NaN     NaN  
3        NaN   

NameError: name 'df_merge' is not defined

In [ ]:
# Create a new DataFrame with the merged columns
df_merge = pd.DataFrame()

# Merge the location columns
df_merge['Location'] = df_merge_sensors['Location_x'].fillna(df_merge_sensors['Location_y'])

# Merge the type columns
df_merge['Type'] = df_merge_sensors['Type_x'].fillna(df_merge_sensors['Type_y'])

# Merge the place columns
df_merge['Place'] = df_merge_sensors['Place_x'].fillna(df_merge_sensors['Place_y'])

# Merge the activity columns
df_merge['Activity'] = df_merge_labels['Activity_x'].fillna(df_merge_labels['Activity_y'])

# Merge the Start time columns
df_merge['Start time'] = df_merge_labels['Start time'].fillna(df_merge_labels['Start time'])

# Merge the End time columns
df_merge['End time'] = df_merge_labels['End time'].fillna(df_merge_labels['End time'])

# Print the first few rows of the merged DataFrame
print(df_merge.head())
print(df_merge.columns)

#df_merge.to_csv('merged_data.csv', index=False)
df_merge.shape

In [ ]:
# Convert 'Start time' and 'End time' columns to datetime format
df_merge['Start time'] = pd.to_datetime(df_merge['Start time'])
df_merge['End time'] = pd.to_datetime(df_merge['End time'])

# Convert 'Location', 'Type', 'Place', and 'Activity' columns to categorical format
df_merge['Location'] = df_merge['Location'].astype('category')
df_merge['Type'] = df_merge['Type'].astype('category')
df_merge['Place'] = df_merge['Place'].astype('category')
df_merge['Activity'] = df_merge['Activity'].astype('category')

df_merge.to_csv('merged_data.csv', index=False)
df = pd.read_csv('merged_data.csv')
print(df.shape)
df.info()
num_cols = ['Location', 'Type', 'Place','Activity','Start time','End time']
#df[num_cols].isnull().sum()

In [ ]:
#Approach 1(Using Standardscalar)
df = pd.read_csv('merged_data.csv')
df = df.drop(['Activity', 'Start time', 'End time'], axis=1)
df = pd.get_dummies(df)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
kmeans = KMeans(n_clusters=11, random_state=42)
kmeans.fit(df_scaled)

In [ ]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(df_scaled)

plt.scatter(principal_components[:,0], principal_components[:,1], c=kmeans.labels_.astype(float))
plt.title('KMeans Clusters')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
##Approach 2(Using Label Encoder)
# Load the dataset
df = pd.read_csv('merged_data.csv')

# Drop the activity column and the time columns
df = df.drop(['Activity', 'Start time', 'End time'], axis=1)

# Convert categorical variables to numeric using label encoding
le = LabelEncoder()
for column in df.columns:
    df[column] = le.fit_transform(df[column])

# Apply k-modes clustering with k=11
km = KModes(n_clusters=11, init='Huang', n_init=5, verbose=1)
clusters = km.fit_predict(df)

# Print the cluster centroids
print(km.cluster_centroids_)

# Calculate the overall accuracy
accuracy = sum(clusters == df.values.argmax(axis=1)) / len(df)
print("Overall accuracy:", accuracy)

# Evaluate the cluster performance using the silhouette coefficient
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(df, clusters)
print("Silhouette coefficient:", silhouette_avg)

# Compute the confusion matrix
from sklearn.metrics import confusion_matrix
labels_true = df.values.argmax(axis=1)
confusion = confusion_matrix(labels_true, clusters)
print("Confusion matrix:")
print(confusion)


In [ ]:
# Add the cluster labels to the original dataframe
df['cluster'] = clusters

# Visualize the clusters
sns.scatterplot(x='Location', y='Type', hue='cluster', data=df)
plt.scatter(km.cluster_centroids_[:, 0], km.cluster_centroids_[:, 1], c='red', marker='x', s=200)
plt.show()


In [ ]:
# Evaluate the cluster performance
hcv = homogeneity_completeness_v_measure(true_labels, clusters)
print(f"Homogeneity: {hcv[0]:.3f}")
print(f"Completeness: {hcv[1]:.3f}")
print(f"V-Measure: {hcv[2]:.3f}")

In [ ]:
#Practice plot
# Load the dataset
df = pd.read_csv('merged_data.csv')

# Drop Activity, Start time and End time columns
df = df.drop(['Activity', 'Start time', 'End time'], axis=1)

# Convert remaining columns to categorical data type
df = df.astype('category')

# Apply kmode clustering with 11 clusters
km = KModes(n_clusters=11, init='Huang', n_init=5, verbose=1)
clusters = km.fit_predict(df)

# Visualize the clusters
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
scatter = ax.scatter(df['Location'], df['Type'], c=clusters, s=50)
legend = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Clusters")
ax.add_artist(legend)
plt.xlabel('Location')
plt.ylabel('Type')
plt.show()